In [1]:
import os
import json
import pandas as pd

In [2]:
# directory = 'data/2024'

# for filename in os.listdir(directory):
#     if filename.endswith('.json'):
#         filepath = os.path.join(directory, filename)
#         with open(filepath, 'r') as file:
#             data = json.load(file)
#             # print(f"Contents of {filename}:")
#             print(data['awd_istr_txt'])
#             print(data['awd_titl_txt'])
#             print(data['abst_narr_txt'])
#             print(data['org_long_name'])
#             print(data['org_long_name2'])
#             print(len(data['pi']))
#             print(data['pi'][0]['proj_role_code2'])
#             print(data['pi'][0]['pi_full_name'])
#             print(data['perf_inst']['perf_inst_name'])
#             print(data['pgm_ele'][0]['pgm_ele_long_name'])
#             print(data['pgm_ref'][0]['pgm_ref_long_name'])
#             # print(json.dumps(data, indent=4))
#     break

In [4]:
import os
import json
import pandas as pd

data_directory = 'data/ranking_data/'
records = []

def safe_get(data, keys, default=None):
    """
    Safely get a nested key from a dictionary using a list of keys.
    Returns default if any key is missing.
    """
    for key in keys:
        if isinstance(data, dict) and key in data:
            data = data[key]
        else:
            return default
    return data

for sub_dir in os.listdir(data_directory):
    print(f"Reading files in {sub_dir}...")
    sub_directory = os.path.join(data_directory, sub_dir)
    for filename in os.listdir(sub_directory):
        if filename.endswith('.json'):
            filepath = os.path.join(sub_directory, filename)
            try:
                with open(filepath, 'r') as file:
                    data = json.load(file)
            except Exception as e:
                print(f"Error reading {filepath}: {e}")
                continue

            # Extract award-level context information safely
            award_type = data.get("awd_istr_txt")
            award_title = data.get("awd_titl_txt")
            abstract = data.get("abst_narr_txt")
            org_name = data.get("org_long_name")
            org_name2 = data.get("org_long_name2")
            perf_inst_name = safe_get(data, ["perf_inst", "perf_inst_name"])
            
            # Extract program element and reference safely (checking if list exists)
            pgm_ele_list = data.get("pgm_ele")
            if isinstance(pgm_ele_list, list) and len(pgm_ele_list) > 0:
                program_element = pgm_ele_list[0].get("pgm_ele_long_name")
            else:
                program_element = None

            pgm_ref_list = data.get("pgm_ref")
            if isinstance(pgm_ref_list, list) and len(pgm_ref_list) > 0:
                program_reference = pgm_ref_list[0].get("pgm_ref_long_name")
            else:
                program_reference = None

            # Get investigator information, ensuring it's a list
            pi_list = data.get("pi")
            if not isinstance(pi_list, list):
                continue

            # Loop through each investigator in the file
            for pi in pi_list:
                record = {
                    "award_type": award_type,
                    "award_title": award_title,
                    "abstract": abstract,
                    "org_name": org_name,
                    "org_name2": org_name2,
                    "perf_inst_name": perf_inst_name,
                    "program_element": program_element,
                    "program_reference": program_reference,
                    "pi_id": pi.get("pi_id"),
                    "pi_full_name": pi.get("pi_full_name", "").strip() if pi.get("pi_full_name") else None,
                    "role": pi.get("proj_role_code2", "").strip() if pi.get("proj_role_code2") else None,
                    "department": pi.get("pi_dept_name"),
                    "email": pi.get("pi_email_addr"),
                    "start_date": pi.get("start_date")
                }
                records.append(record)

# Create a DataFrame from the records
df = pd.DataFrame(records)


Reading files in 2022...
Reading files in 2024...
Reading files in 2023...
Reading files in 2021...
Reading files in 2020...


In [ ]:
# df = pd.read_csv('data/ranking_data/nsf_dataset.csv')
# print(df.shape)
# df.head()

In [5]:
df.role.value_counts()

role
Principal Investigator              50094
Co-Principal Investigator           33018
Former Co-Principal Investigator     3276
Former Principal Investigator        2658
Name: count, dtype: int64

In [6]:
df = df[df['role'].isin(['Co-Principal Investigator', 'Principal Investigator'])]
df.head()

,award_type,award_title,abstract,org_name,org_name2,perf_inst_name,program_element,program_reference,pi_id,pi_full_name,role,department,email,start_date
0,Standard Grant,MRI: Acquisition of a High-Performance Computa...,This project will acquire and deploy a high-pe...,Directorate for Computer and Information Scien...,Office of Advanced Cyberinfrastructure (OAC),Oklahoma State University,Major Research Instrumentation,"WOMEN, MINORITY, DISABLED, NEC",269967889,Terrance Figy,Co-Principal Investigator,"Mathematics, Statistics, and Physics",Terrance.Figy@wichita.edu,2024-08-29
1,Standard Grant,MRI: Acquisition of a High-Performance Computa...,This project will acquire and deploy a high-pe...,Directorate for Computer and Information Scien...,Office of Advanced Cyberinfrastructure (OAC),Oklahoma State University,Major Research Instrumentation,"WOMEN, MINORITY, DISABLED, NEC",269758255,Pratul K Agarwal,Principal Investigator,,pratul.agarwal@okstate.edu,2022-08-03
2,Standard Grant,MRI: Acquisition of a High-Performance Computa...,This project will acquire and deploy a high-pe...,Directorate for Computer and Information Scien...,Office of Advanced Cyberinfrastructure (OAC),Oklahoma State University,Major Research Instrumentation,"WOMEN, MINORITY, DISABLED, NEC",000224099,Mickey Slimp,Co-Principal Investigator,Department of Chemistry,,2024-08-29
4,Standard Grant,MRI: Acquisition of a High-Performance Computa...,This project will acquire and deploy a high-pe...,Directorate for Computer and Information Scien...,Office of Advanced Cyberinfrastructure (OAC),Oklahoma State University,Major Research Instrumentation,"WOMEN, MINORITY, DISABLED, NEC",269666332,William H Hsu,Co-Principal Investigator,Computer Science,bhsu@ksu.edu,2022-08-03
7,Standard Grant,MRI: Acquisition of a High-Performance Computa...,This project will acquire and deploy a high-pe...,Directorate for Computer and Information Scien...,Office of Advanced Cyberinfrastructure (OAC),Oklahoma State University,Major Research Instrumentation,"WOMEN, MINORITY, DISABLED, NEC",270046494,Robert Fleming,Co-Principal Investigator,Engineering,rofleming@AState.edu,2024-08-29


In [7]:
len(df.pi_id.unique().tolist()), len(df.pi_full_name.unique().tolist())

(54006, 56701)

In [8]:
df.columns

Index(['award_type', 'award_title', 'abstract', 'org_name', 'org_name2',
       'perf_inst_name', 'program_element', 'program_reference', 'pi_id',
       'pi_full_name', 'role', 'department', 'email', 'start_date'],
      dtype='object')

In [32]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Combine relevant text columns into one (you may adjust columns as needed)
text_columns = [
    "award_type", "award_title", "abstract", 
    "org_name", "org_name2", "perf_inst_name", 
    "program_element", "program_reference"
]
df["combined_text"] = df[text_columns].astype(str).agg(" ".join, axis=1)

# a. Leadership indicator: 1 if role suggests prior leadership (e.g., contains "Principal Investigator")
df["leadership"] = df["role"].apply(lambda x: 1 if "Principal Investigator" in str(x) else 0)

# b. Experience in years: use start_date and a reference date (here we use today)
df["start_date"] = pd.to_datetime(df["start_date"], errors='coerce')
reference_date = datetime.now()  # or use a fixed project date
df["experience_years"] = (reference_date - df["start_date"]).dt.days / 365.25

# Load a pre-trained sentence transformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Compute embedding for each award's combined text
df["text_embedding"] = df["combined_text"].apply(lambda x: embedder.encode(x))

# We assume each row has a researcher ID ("pi_id"). If a researcher has multiple rows, we aggregate.
# For aggregated text, we average the embeddings; for numeric features, we use appropriate aggregation.
award_counts = df.groupby("pi_id").size().reset_index(name="award_count")
df_grouped = df.groupby("pi_id").agg({
    "experience_years": "mean",       # average experience across awards
    "leadership": "max",              # if they have ever been a PI, mark as leadership
    "text_embedding": lambda embs: np.mean(np.stack(embs), axis=0)
}).reset_index()
df_grouped = df_grouped.merge(award_counts, on="pi_id", how="left")

# For later scoring, normalize the numeric features (experience and award_count)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df_grouped[["exp_norm", "award_norm"]] = scaler.fit_transform(df_grouped[["experience_years", "award_count"]])

def rank_candidates(research_topic, candidate_ids, base_weight=0.5, topic_weight=0.5):
    # Compute embedding for the research topic
    topic_emb = embedder.encode(research_topic)
    
    candidate_scores = []
    for cid in candidate_ids:
        candidate = df_grouped[df_grouped["pi_id"] == cid].iloc[0]
        
        # Topic relevance score: cosine similarity between candidate's aggregated embedding and the topic
        candidate_emb = candidate["text_embedding"]
        relevance_score = cosine_similarity([candidate_emb], [topic_emb])[0][0]
        
        # Base score: a simple weighted sum of normalized features plus a bonus for leadership
        # Adjust weights as needed. Here, leadership gets a bonus of 1 if present.
        base_score = candidate["exp_norm"] + candidate["award_norm"] + (1 if candidate["leadership"] == 1 else 0)
        
        # Combined score: weighted combination of base score and topic relevance
        combined_score = base_weight * base_score + topic_weight * relevance_score
        candidate_scores.append(combined_score)
    
    candidate_scores = np.array(candidate_scores)
    best_index = np.argmax(candidate_scores)
    pi_candidate = candidate_ids[best_index]
    co_pi_candidates = [cid for i, cid in enumerate(candidate_ids) if i != best_index]
    
    return pi_candidate, co_pi_candidates, candidate_scores

In [33]:
df_grouped.head()

,pi_id,experience_years,leadership,text_embedding,award_count,exp_norm,award_norm
0,000000084,1.468857,1,"[0.0027017621, 0.026450962, 0.04872246, 0.0352...",2,0.262109,0.028571
1,000000094,2.904860,1,"[-0.026076434, -0.012992876, -0.023296108, 0.0...",1,0.526741,0.000000
2,000000132,3.676934,1,"[-0.0791296, 0.043611363, -0.075964525, 0.0038...",1,0.669021,0.000000
3,000000390,2.201232,1,"[-0.09878656, -0.029403467, 0.008581905, 0.061...",1,0.397074,0.000000
4,000000453,2.151951,1,"[-0.064013764, -0.05129957, -0.045931555, 0.03...",2,0.387992,0.028571


In [38]:
df_grouped.to_csv('data/ranking_data/final_nsf_dataset.csv', index=False)

In [34]:
candidate_ids = ['000025017', '000025762', '000030655']

In [ ]:
# Filter the dataframe to get rows where pi_id is in candidate_ids
filtered_df = df[df['pi_id'].isin(candidate_ids)]
filtered_df

In [35]:
df.program_element.unique()

array(['Major Research Instrumentation', 'Cellular Dynamics and Function',
       'SBIR Outreach & Tech. Assist', 'CONDENSED MATTER PHYSICS',
       'EchemS-Electrochemical Systems', 'Geobiology & Low-Temp Geochem',
       'Biology Postdoctoral Research', 'I-Corps',
       'Postdoctoral Fellowships', 'Cellular & Biochem Engineering',
       'Integrat & Collab Ed & Rsearch', 'IUSE',
       'EPMD-ElectrnPhoton&MagnDevices', 'D-ISN-Illicit Supply Networks',
       'EDSE-Engineering Design and Systems Engineering',
       'Graduate Research Fellowship', 'Physical & Dynamic Meteorology',
       'Molecular Biophysics', 'Discovery Research K-12',
       'Gravity Theory', 'Robust Intelligence', 'ADVANCE',
       'ER2-Ethical & Responsible Res', 'FW-HTF Futr Wrk Hum-Tech Frntr',
       'BOCP-Biodiv on Changing Planet', 'Paleoclimate',
       'OFFICE OF MULTIDISCIPLINARY AC',
       'AMPS-Algorithms for Modern Power Systems',
       'EPSCoR RII Track-4: Res Fellow', 'Archaeology',
       'INFRAS

In [ ]:
[n for n in df.program_element.unique() if n is not None and 'biom' in str(n)]


In [12]:
# df[df.program_element == 'AI-Safety']
# 269963391, 269911282

In [36]:
research_topic = "STATISTICS"  # can be any topic, e.g., "confidential computing"
# candidate_ids = ['000025017', '000025762', '000030655']
candidate_ids = ['269963391', '269911282', '000025762', '269878000']
pi_candidate, co_pi_candidates, scores = rank_candidates(research_topic, candidate_ids)
print("Predicted PI:", pi_candidate)
print("Predicted Co-PIs:", co_pi_candidates)
print("Candidate Combined Scores:", scores)

Predicted PI: 269911282
Predicted Co-PIs: ['269963391', '000025762', '269878000']
Candidate Combined Scores: [0.88965248 1.01012845 0.92949853 0.83147678]


In [37]:
# Test with various topics dynamically.
for test_topic in ["knowledge graph", "AI", "Neuroscience", "STATISTICS"]:
    print("Testing with topic:", test_topic)
    pi_candidate, co_pi_candidates, scores = rank_candidates(test_topic, candidate_ids)
    print("Predicted PI:", pi_candidate)
    print("Predicted Co-PIs:", co_pi_candidates)
    print("Candidate Combined Scores:", scores)
    print("-" * 50)

Testing with topic: knowledge graph
Predicted PI: 269911282
Predicted Co-PIs: ['269963391', '000025762', '269878000']
Candidate Combined Scores: [0.97457155 1.11437134 0.88944828 0.79709446]
--------------------------------------------------
Testing with topic: AI
Predicted PI: 269963391
Predicted Co-PIs: ['269911282', '000025762', '269878000']
Candidate Combined Scores: [1.10800765 1.02881144 0.88151991 0.83231751]
--------------------------------------------------
Testing with topic: Neuroscience
Predicted PI: 269878000
Predicted Co-PIs: ['269963391', '269911282', '000025762']
Candidate Combined Scores: [0.95705275 0.97001489 0.8349548  1.00054973]
--------------------------------------------------
Testing with topic: STATISTICS
Predicted PI: 269911282
Predicted Co-PIs: ['269963391', '000025762', '269878000']
Candidate Combined Scores: [0.88965248 1.01012845 0.92949853 0.83147678]
--------------------------------------------------


In [1]:
df[df.pi_id.isin(candidate_ids)][['pi_id', 'role', 'department', 'leadership', 'experience_years', 'program_element']]

NameError: name 'df' is not defined

In [ ]:
candidate_ids

In [ ]:
df_grouped[df_grouped.pi_id.isin(candidate_ids)]

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import xgboost as xgb
from sklearn.utils import shuffle

# ================================
# Step 1: Load the Dataset
# ================================
# Replace "your_dataset.csv" with your actual CSV file.
# df = pd.read_csv("your_dataset.csv")

# ================================
# Step 2: Combine Text Fields for Each Award
# ================================
text_columns = [
    "award_type", "award_title", "abstract", 
    "org_name", "org_name2", "perf_inst_name", 
    "program_element", "program_reference"
]
df["combined_text"] = df[text_columns].astype(str).agg(" ".join, axis=1)

# ================================
# Step 3: Compute Basic Award-Level Features
# ================================
# (a) Leadership flag: 1 if role indicates prior leadership (e.g., "Principal Investigator")
# df["leadership"] = df["role"].apply(lambda x: 1 if "Principal Investigator" in str(x) else 0)
df["leadership"] = df.groupby("pi_id")["role"].transform(lambda x: (x == "Principal Investigator").sum())

# (b) Experience (in years): using start_date and a reference date (today)
df["start_date"] = pd.to_datetime(df["start_date"], errors='coerce')
reference_date = datetime.now()  # or use a fixed project date
df["experience_years"] = (reference_date - df["start_date"]).dt.days / 365.25

# ================================
# Step 4: Compute Text Embeddings for Each Award
# ================================
# Load a pre-trained sentence transformer model.
embedder = SentenceTransformer("all-MiniLM-L6-v2")
df["text_embedding"] = df["combined_text"].apply(lambda x: embedder.encode(x))

# ================================
# Step 5: Aggregate Features Per Researcher
# ================================
# For each researcher (identified by "pi_id"), we compute:
# - Maximum experience (i.e. the highest years value among awards)
# - Maximum leadership flag (if they ever held a leadership role)
# - Award count
# - Aggregated text embedding (average of the embeddings from their awards)
award_counts = df.groupby("pi_id").size().reset_index(name="award_count")

df_grouped = df.groupby("pi_id").agg({
    "experience_years": "max",
    "leadership": "max",
    "text_embedding": lambda embs: np.mean(np.stack(embs), axis=0)
}).reset_index()

df_grouped = df_grouped.merge(award_counts, on="pi_id", how="left")

# ================================
# Step 6: Normalize Numerical Features
# ================================
scaler = MinMaxScaler()
df_grouped[["exp_norm", "award_norm"]] = scaler.fit_transform(
    df_grouped[["experience_years", "award_count"]]
)

# ================================
# Step 7: Compute a Heuristic Base Score (For Simulation)
# ================================
# Base score is a simple weighted sum of normalized experience, normalized award count,
# and a bonus for leadership.
df_grouped["base_score"] = df_grouped["exp_norm"] + df_grouped["award_norm"] + df_grouped["leadership"]

# ================================
# Step 8: Compute Topic Relevance for a Fixed Training Topic
# ================================
# For training, we simulate a fixed research topic, e.g. "knowledge graph".
fixed_topic = "knowledge graph"
fixed_topic_emb = embedder.encode(fixed_topic)

def compute_topic_relevance(emb):
    return cosine_similarity([emb], [fixed_topic_emb])[0][0]

df_grouped["topic_relevance"] = df_grouped["text_embedding"].apply(compute_topic_relevance)

# ================================
# Step 9: Construct Training Groups (Simulated Candidate Pools)
# ================================
# We simulate candidate groups by randomly grouping researchers (e.g., groups of 3)
df_grouped = shuffle(df_grouped, random_state=42).reset_index(drop=True)
group_size = 3
num_groups = len(df_grouped) // group_size
group_ids = np.repeat(np.arange(num_groups), group_size)
if len(df_grouped) % group_size != 0:
    group_ids = np.concatenate([group_ids, np.full(len(df_grouped) % group_size, num_groups)])
df_grouped["group_id"] = group_ids

# ================================
# Step 10: Create Training Labels for Ranking
# ================================
# Within each group, assign label 1 to the candidate with the highest base_score,
# and 0 to the others.
df_grouped["label"] = 0
for group in df_grouped["group_id"].unique():
    group_mask = df_grouped["group_id"] == group
    if group_mask.sum() > 0:
        best_index = df_grouped.loc[group_mask, "base_score"].idxmax()
        df_grouped.loc[best_index, "label"] = 1

# ================================
# Step 11: Prepare Data for XGBoost Ranking Model
# ================================
# Use features: normalized experience, normalized award count, leadership, and topic_relevance.
features = ["exp_norm", "award_norm", "leadership", "topic_relevance"]
X = df_grouped[features].values
y = df_grouped["label"].values

# XGBoost ranking requires group information: number of candidates in each group.
group_data = df_grouped.groupby("group_id").size().tolist()

dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group_data)

# ================================
# Step 12: Train the XGBoost Ranking Model
# ================================
params = {
    "objective": "rank:pairwise",
    "eval_metric": "ndcg",
    "eta": 0.1,
    "max_depth": 3,
    "seed": 42
}
num_round = 50
rank_model = xgb.train(params, dtrain, num_boost_round=num_round)

# ================================
# Step 13: Define an Inference Function for PI Selection
# ================================
def predict_pi(candidate_ids, research_topic):
    """
    Given a list of candidate researcher IDs and a research topic,
    compute each candidate's features (including topic relevance computed against the provided topic),
    predict a ranking score using the trained ranking model, and return candidates ranked in descending order.
    
    Parameters:
      candidate_ids (list): List of candidate pi_id values.
      research_topic (str): The research topic (e.g., "confidential computing").
    
    Returns:
      ranked_candidates (list): Candidate pi_ids ordered from best (highest score) to worst.
      scores (np.array): Predicted ranking scores for each candidate.
    """
    topic_emb = embedder.encode(research_topic)
    feature_list = []
    
    # For each candidate, extract the pre-computed normalized features and recompute topic relevance.
    for cid in candidate_ids:
        candidate_row = df_grouped[df_grouped["pi_id"] == cid]
        if candidate_row.empty:
            continue
        candidate = candidate_row.iloc[0]
        # Recompute topic relevance based on the new research topic
        topic_rel = cosine_similarity([candidate["text_embedding"]], [topic_emb])[0][0]
        feature_list.append([
            candidate["exp_norm"],
            candidate["award_norm"],
            candidate["leadership"],
            topic_rel
        ])
    if len(feature_list) == 0:
        return None, None
    X_new = np.array(feature_list)
    dtest = xgb.DMatrix(X_new)
    preds = rank_model.predict(dtest)
    # Rank candidates in descending order (best candidate first)
    ranked_indices = np.argsort(-preds)
    ranked_candidates = [candidate_ids[i] for i in ranked_indices]
    return ranked_candidates, preds


In [ ]:
candidate_ids = ['000025017', '000025762', '000030655']
research_topic = "STATISTICS"

In [ ]:
# ================================
# Step 14: Example Usage
# ================================
# Suppose you have a set of candidate researchers (e.g., from your aggregated dataset)
# For demonstration, we randomly pick 4 candidate pi_ids.
# candidate_ids = df_grouped["pi_id"].sample(4, random_state=42).tolist()
# research_topic = "confidential computing"  # This can be any topic of interest

ranked_candidates, scores = predict_pi(candidate_ids, research_topic)
print("Candidate IDs:", candidate_ids)
print("Ranked Candidates (best first):", ranked_candidates)
print("Ranking Scores:", scores)

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import xgboost as xgb
from sklearn.utils import shuffle

# ================================
# Step 1: Load the Dataset
# ================================
# df = pd.read_csv("your_dataset.csv")

# ================================
# Step 2: Combine Text Fields for Each Award
# ================================
text_columns = [
    "award_type", "award_title", "abstract", 
    "org_name", "org_name2", "perf_inst_name", 
    "program_element", "program_reference"
]
df["combined_text"] = df[text_columns].astype(str).agg(" ".join, axis=1)

# ================================
# Step 3: Compute Award-Level Features
# ================================
# (a) Leadership flag: 1 if role indicates prior leadership (e.g., "Principal Investigator")
df["leadership"] = df["role"].apply(lambda x: 1 if "Principal Investigator" in str(x) else 0)

# (b) Experience (in years): use start_date and a reference date (here, we use current date)
df["start_date"] = pd.to_datetime(df["start_date"], errors='coerce')
reference_date = datetime.now()
df["experience_years"] = (reference_date - df["start_date"]).dt.days / 365.25

# ================================
# Step 4: Compute Text Embeddings for Each Award
# ================================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
df["text_embedding"] = df["combined_text"].apply(lambda x: embedder.encode(x))

# ================================
# Step 5: Aggregate Features Per Researcher
# ================================
# For each researcher (identified by "pi_id"), we compute:
# - Maximum experience (i.e. the highest years value among awards)
# - Maximum leadership flag (if they ever held a leadership role)
# - Award count (number of awards)
# - Aggregated text embedding (average of the embeddings from their awards)
award_counts = df.groupby("pi_id").size().reset_index(name="award_count")
df_grouped = df.groupby("pi_id").agg({
    "experience_years": "max",
    "leadership": "max",
    "text_embedding": lambda embs: np.mean(np.stack(embs), axis=0)
}).reset_index()
df_grouped = df_grouped.merge(award_counts, on="pi_id", how="left")

# ================================
# Step 6: Normalize Numerical Features
# ================================
scaler = MinMaxScaler()
df_grouped[["exp_norm", "award_norm"]] = scaler.fit_transform(
    df_grouped[["experience_years", "award_count"]]
)

# ================================
# Step 7: Compute a Base Score (Independent of Topic)
# ================================
# For simulation, we use a simple weighted sum of normalized experience and award count,
# plus a bonus if the researcher has leadership experience.
df_grouped["base_score"] = df_grouped["exp_norm"] + df_grouped["award_norm"] + df_grouped["leadership"]

# ================================
# Step 8: Simulate Dynamic Topics During Training
# ================================
# Define a list of possible topics (these can be any research areas)
topics_list = ["knowledge graph", "confidential computing", "machine learning", "data mining", "cybersecurity"]

# For simulation, we will form candidate groups and assign each group a random topic.
df_grouped = shuffle(df_grouped, random_state=42).reset_index(drop=True)
group_size = 3  # or any group size representing candidate pools
num_groups = len(df_grouped) // group_size
group_ids = np.repeat(np.arange(num_groups), group_size)
if len(df_grouped) % group_size != 0:
    group_ids = np.concatenate([group_ids, np.full(len(df_grouped) % group_size, num_groups)])
df_grouped["group_id"] = group_ids

# For each group, assign a random topic and compute dynamic topic relevance for each candidate.
# We also simulate the "true" ranking label based on a weighted sum of base_score and dynamic topic relevance.
base_score_weight = 0.5
topic_weight = 0.5

# Prepare a column to hold the dynamic topic relevance
df_grouped["topic_relevance"] = 0.0
df_grouped["assigned_topic"] = ""

# Process each group
for group in df_grouped["group_id"].unique():
    group_mask = df_grouped["group_id"] == group
    # Assign a random topic from topics_list to this group
    dynamic_topic = np.random.choice(topics_list)
    df_grouped.loc[group_mask, "assigned_topic"] = dynamic_topic
    topic_emb = embedder.encode(dynamic_topic)
    
    # Compute topic relevance for each candidate in the group based on cosine similarity
    relevances = []
    for idx in df_grouped.loc[group_mask].index:
        candidate_emb = df_grouped.at[idx, "text_embedding"]
        relevance = cosine_similarity([candidate_emb], [topic_emb])[0][0]
        relevances.append(relevance)
    df_grouped.loc[group_mask, "topic_relevance"] = relevances
    
    # Compute a simulated combined score: weighted sum of base_score and dynamic topic relevance.
    combined_scores = base_score_weight * df_grouped.loc[group_mask, "base_score"] + topic_weight * df_grouped.loc[group_mask, "topic_relevance"]
    # Assign label 1 to the candidate with the highest combined score; 0 for others.
    best_idx = combined_scores.idxmax()
    df_grouped.loc[group_mask, "label"] = 0  # initialize
    df_grouped.at[best_idx, "label"] = 1

# ================================
# Step 9: Prepare Data for XGBoost Ranking Model
# ================================
# Our training features include: normalized experience, normalized award count, leadership flag, and dynamic topic_relevance.
features = ["exp_norm", "award_norm", "leadership", "topic_relevance"]
X = df_grouped[features].values
y = df_grouped["label"].values

# XGBoost ranking requires group information (number of candidates per group)
group_data = df_grouped.groupby("group_id").size().tolist()

dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group_data)

# ================================
# Step 10: Train the XGBoost Ranking Model
# ================================
params = {
    "objective": "rank:pairwise",
    "eval_metric": "ndcg",
    "eta": 0.1,
    "max_depth": 3,
    "seed": 42
}
num_round = 50
rank_model = xgb.train(params, dtrain, num_boost_round=num_round)

# ================================
# Step 11: Define an Inference Function for Dynamic Topic Ranking
# ================================
def predict_pi(candidate_ids, research_topic):
    """
    Given a list of candidate researcher IDs and a research topic,
    compute each candidate's dynamic topic relevance and then predict ranking scores
    using the trained ranking model. Returns candidates ranked in descending order.
    
    Parameters:
      candidate_ids (list): List of candidate pi_id values.
      research_topic (str): The research topic (e.g., "confidential computing").
    
    Returns:
      ranked_candidates (list): Candidate pi_ids ordered from best (highest score) to worst.
      scores (np.array): Predicted ranking scores for each candidate.
    """
    topic_emb = embedder.encode(research_topic)
    feature_list = []
    
    # For each candidate, extract pre-computed normalized features and recompute topic relevance using the new topic.
    for cid in candidate_ids:
        candidate_row = df_grouped[df_grouped["pi_id"] == cid]
        if candidate_row.empty:
            continue
        candidate = candidate_row.iloc[0]
        # Recompute topic relevance for the provided research_topic
        topic_rel = cosine_similarity([candidate["text_embedding"]], [topic_emb])[0][0]
        feature_list.append([
            candidate["exp_norm"],
            candidate["award_norm"],
            candidate["leadership"],
            topic_rel
        ])
    if len(feature_list) == 0:
        return None, None
    X_new = np.array(feature_list)
    dtest = xgb.DMatrix(X_new)
    preds = rank_model.predict(dtest)
    ranked_indices = np.argsort(-preds)  # descending order
    ranked_candidates = [candidate_ids[i] for i in ranked_indices]
    return ranked_candidates, preds




In [ ]:
candidate_ids = ['000025017', '000025762', '000030655']
research_topic = "STATISTICS"

ranked_candidates, scores = predict_pi(candidate_ids, research_topic)
print("Candidate IDs:", candidate_ids)
print("Ranked Candidates (best first):", ranked_candidates)
print("Ranking Scores:", scores)

In [ ]:
# ================================
# Step 12: Example Usage
# ================================
# Here we simulate a few candidate groups with dynamic topics.
# For example, we pick candidate lists and provide a research topic.
candidate_ids_example = df_grouped["pi_id"].sample(4, random_state=42).tolist()

# Now test with various topics dynamically:
for test_topic in ["knowledge graph", "confidential computing", "data mining", 'STATISTICS']:
    ranked_candidates, scores = predict_pi(candidate_ids_example, test_topic)
    print("Research Topic:", test_topic)
    print("Candidate IDs:", candidate_ids_example)
    print("Ranked Candidates (best first):", ranked_candidates)
    print("Ranking Scores:", scores)
    print("-" * 50)

In [ ]:
df[df.pi_id.isin(candidate_ids_example)][['pi_id', 'role', 'department', 'leadership', 'experience_years']]

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import xgboost as xgb
from sklearn.utils import shuffle


# ================================
# Step 2: Combine Text Fields for Each Award
# ================================
text_columns = [
    "award_type", "award_title", "abstract", 
    "org_name", "org_name2", "perf_inst_name", 
    "program_element", "program_reference"
]
df["combined_text"] = df[text_columns].astype(str).agg(" ".join, axis=1)

# ================================
# Step 3: Compute Award-Level Features
# ================================
# Leadership flag: 1 if role suggests prior leadership (e.g., "Principal Investigator")
# df["leadership"] = df["role"].apply(lambda x: 1 if "Principal Investigator" in str(x) else 0)
df["leadership"] = df.groupby("pi_id")["role"].transform(lambda x: (x == "Principal Investigator").sum())

# Experience (in years) using start_date and current date
df["start_date"] = pd.to_datetime(df["start_date"], errors='coerce')
reference_date = datetime.now()
df["experience_years"] = (reference_date - df["start_date"]).dt.days / 365.25

# ================================
# Step 4: Compute Text Embeddings for Each Award
# ================================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
df["text_embedding"] = df["combined_text"].apply(lambda x: embedder.encode(x))

# ================================
# Step 5: Aggregate Features Per Researcher
# ================================
# For each researcher (identified by "pi_id"), aggregate features:
award_counts = df.groupby("pi_id").size().reset_index(name="award_count")
df_grouped = df.groupby("pi_id").agg({
    "experience_years": "max",   # best (largest) experience across awards
    "leadership": "max",         # if ever a PI, flag as 1
    "text_embedding": lambda embs: np.mean(np.stack(embs), axis=0)
}).reset_index()
df_grouped = df_grouped.merge(award_counts, on="pi_id", how="left")

# Also bring in department information (assuming each researcher has a unique department)
# If a researcher has multiple departments, choose one or aggregate appropriately.
dept_info = df.groupby("pi_id")["department"].first().reset_index()
df_grouped = df_grouped.merge(dept_info, on="pi_id", how="left")

# ================================
# Step 6: Normalize Numerical Features
# ================================
scaler = MinMaxScaler()
df_grouped[["exp_norm", "award_norm"]] = scaler.fit_transform(
    df_grouped[["experience_years", "award_count"]]
)

# ================================
# Step 7: Compute a Base Score (Independent of Topic)
# ================================
# Simple weighted sum: experience, award count, and leadership bonus.
df_grouped["base_score"] = df_grouped["exp_norm"] + df_grouped["award_norm"] + df_grouped["leadership"]

# ================================
# Step 8: Incorporate Department Information
# ================================
# One-hot encode the 'department' column.
dept_dummies = pd.get_dummies(df_grouped["department"], prefix="dept")
df_grouped = pd.concat([df_grouped, dept_dummies], axis=1)
# Get a list of department feature names.
department_features = list(dept_dummies.columns)

# ================================
# Step 9: Simulate Dynamic Topics During Training
# ================================
# Define a list of topics (these are example research areas).
topics_list = ["knowledge graph", "confidential computing", "machine learning", "data mining", "cybersecurity", "statistics"]
df_grouped = shuffle(df_grouped, random_state=42).reset_index(drop=True)
group_size = 3  # candidate pool size
num_groups = len(df_grouped) // group_size
group_ids = np.repeat(np.arange(num_groups), group_size)
if len(df_grouped) % group_size != 0:
    group_ids = np.concatenate([group_ids, np.full(len(df_grouped) % group_size, num_groups)])
df_grouped["group_id"] = group_ids

# Set weights for simulation
base_score_weight = 0.5
topic_weight = 0.5

# Initialize topic relevance and assigned topic columns
df_grouped["topic_relevance"] = 0.0
df_grouped["assigned_topic"] = ""

for group in df_grouped["group_id"].unique():
    group_mask = df_grouped["group_id"] == group
    dynamic_topic = np.random.choice(topics_list)
    df_grouped.loc[group_mask, "assigned_topic"] = dynamic_topic
    topic_emb = embedder.encode(dynamic_topic)
    
    relevances = []
    for idx in df_grouped.loc[group_mask].index:
        candidate_emb = df_grouped.at[idx, "text_embedding"]
        relevance = cosine_similarity([candidate_emb], [topic_emb])[0][0]
        relevances.append(relevance)
    df_grouped.loc[group_mask, "topic_relevance"] = relevances
    
    # Combined score now includes the dynamic topic relevance.
    combined_scores = base_score_weight * df_grouped.loc[group_mask, "base_score"] + topic_weight * df_grouped.loc[group_mask, "topic_relevance"]
    best_idx = combined_scores.idxmax()
    df_grouped.loc[group_mask, "label"] = 0
    df_grouped.at[best_idx, "label"] = 1

# ================================
# Step 10: Prepare Data for XGBoost Ranking Model
# ================================
# Define feature list, now including department one-hot features.
features = ["exp_norm", "award_norm", "leadership", "topic_relevance"] + department_features
X = df_grouped[features].values
y = df_grouped["label"].values

# Group information for ranking.
group_data = df_grouped.groupby("group_id").size().tolist()

dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group_data)

# ================================
# Step 11: Train the XGBoost Ranking Model
# ================================
params = {
    "objective": "rank:pairwise",
    "eval_metric": "ndcg",
    "eta": 0.1,
    "max_depth": 3,
    "seed": 42
}
num_round = 50
rank_model = xgb.train(params, dtrain, num_boost_round=num_round)

# ================================
# Step 12: Inference Function for Dynamic Topic Ranking
# ================================
def predict_pi(candidate_ids, research_topic):
    """
    Given a list of candidate researcher IDs and a research topic,
    compute each candidate's dynamic topic relevance and additional department features,
    then predict ranking scores using the trained ranking model.
    
    Returns:
      ranked_candidates (list): Candidate pi_ids ordered from best to worst.
      scores (np.array): Ranking scores for each candidate.
    """
    topic_emb = embedder.encode(research_topic)
    feature_list = []
    
    # For each candidate, recompute topic relevance with the new research_topic.
    for cid in candidate_ids:
        candidate_row = df_grouped[df_grouped["pi_id"] == cid]
        if candidate_row.empty:
            continue
        candidate = candidate_row.iloc[0]
        topic_rel = cosine_similarity([candidate["text_embedding"]], [topic_emb])[0][0]
        # Build the feature vector: use normalized features and department dummy values.
        dept_features = candidate[department_features].values.tolist()
        features_vector = [
            candidate["exp_norm"],
            candidate["award_norm"],
            candidate["leadership"],
            topic_rel
        ] + dept_features
        feature_list.append(features_vector)
    if len(feature_list) == 0:
        return None, None
    X_new = np.array(feature_list)
    dtest = xgb.DMatrix(X_new)
    preds = rank_model.predict(dtest)
    ranked_indices = np.argsort(-preds)  # sort descending by score
    ranked_candidates = [candidate_ids[i] for i in ranked_indices]
    return ranked_candidates, preds

# ================================
# Step 13: Example Usage with Dynamic Topics
# ================================
# Sample a set of candidate researcher IDs.
candidate_ids_example = df_grouped["pi_id"].sample(4, random_state=42).tolist()

# Test with various topics dynamically.
for test_topic in ["knowledge graph", "confidential computing", "data mining", "STATISTICS"]:
    ranked_candidates, scores = predict_pi(candidate_ids_example, test_topic)
    print("Research Topic:", test_topic)
    print("Candidate IDs:", candidate_ids_example)
    print("Ranked Candidates (best first):", ranked_candidates)
    print("Ranking Scores:", scores)
    print("-" * 50)


In [ ]:
df[df.pi_id.isin(candidate_ids_example)][['pi_id', 'role', 'department', 'leadership', 'experience_years']]

In [ ]:
research_topic = "STATISTICS"  # can be any topic, e.g., "confidential computing"
candidate_ids = ['000025017', '000025762', '000030655']
ranked_candidates, scores = predict_pi(candidate_ids, research_topic)
print("Research Topic:", test_topic)
print("Candidate IDs:", candidate_ids_example)
print("Ranked Candidates (best first):", ranked_candidates)
print("Ranking Scores:", scores)

In [ ]:
df[df.pi_id.isin(candidate_ids)][['pi_id', 'role', 'department', 'leadership', 'experience_years']]